In [1]:
%pwd

'/home/uday/Practise/github_personal/pothole_detection/pothole_detection/research'

In [2]:
import os
os.chdir("../")
%pwd

'/home/uday/Practise/github_personal/pothole_detection/pothole_detection'

In [3]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    unzip_folder: str

In [4]:
from potholeDetector.constants import *
from potholeDetector.utils.common import *

In [5]:
class ConfigurationManager:
    def __init__(
                self,
                config_filepath: str = CONFIG_FILE_PATH,
                params_filepath: str = PARAMS_FILE_PATH):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)

                create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                    root_dir = config.root_dir,
                    source_URL = config.source_URL,
                    local_data_file = config.local_data_file,
                    unzip_dir = config.unzip_dir,
                    unzip_folder = config.unzip_folder,
            )

            return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from potholeDetector import logger
from potholeDetector.utils.common import *

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
         

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory and renames the extracted folder.
        """
        print("i am here")
        unzip_path = self.config.unzip_dir
        unzip_folder = self.config.unzip_folder
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            zip_infos = zip_ref.infolist()
            final_folder_name = zip_infos[0].filename            

        # Get the extracted folder name
        # extracted_folder_name = os.path.splitext(os.path.basename(self.config.local_data_file))[0]
        print(final_folder_name)

        # Get the full path of the extracted folder
        extracted_folder_full_path = os.path.join(unzip_path, final_folder_name)


        # Rename the extracted folder
        renamed_folder_path = os.path.join(unzip_path, unzip_folder)
        print(extracted_folder_full_path,renamed_folder_path)
        os.rename(extracted_folder_full_path, renamed_folder_path)



In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-19 01:10:23,667: INFO:common:YAML file: config/config.yaml loaded successfully]
[2023-07-19 01:10:23,669: INFO:common:YAML file: params.yaml loaded successfully]
[2023-07-19 01:10:23,669: INFO:common:Created directory at: artifacts]
[2023-07-19 01:10:23,670: INFO:common:Created directory at: artifacts/data_ingestion]
[2023-07-19 01:14:06,491: INFO:361887018:artifacts/data_ingestion/data.zip download! with following info: 
Accept-Ranges: bytes
Cache-Control: max-age=60
Content-Disposition: attachment; filename="pothole_dataset_v8.zip"; filename*=UTF-8''pothole_dataset_v8.zip
Content-Security-Policy: sandbox
Etag: 1673237689276858d
Pragma: public
Referrer-Policy: no-referrer
Vary: Origin
X-Content-Security-Policy: sandbox
X-Content-Type-Options: nosniff
X-Robots-Tag: noindex, nofollow, noimageindex
X-Server-Response-Time: 321
X-Webkit-Csp: sandbox
Content-Type: application/binary
Accept-Encoding: identity,gzip
Date: Tue, 18 Jul 2023 19:40:25 GMT
Server: envoy
Strict-Transport-Se